In [23]:
#This script takes a initial date and final date and downloads the files needed to compute the wave activity flux (WAF)
#for that given period. NCEP/NCAR Reanalysis (Kalnay etal 1996)are used 

#libraries needed
import urllib.request
from bs4 import BeautifulSoup
import numpy as np

def clean():   #clean enviroment
    import os
    os.system("rm -f /tmp/*.gz /tmp/*.nc")
    
def descarga_nc( mesi, diai, mesf, diaf, aniof, variable_entrada, variable_salida, tipo):
    #tipo controla que dato bajo: 2: para anomalia 3 para climatologia
    # Open NCEP NCAR to access to lik to data

        url = 'http://www.esrl.noaa.gov/psd/cgi-bin/data/composites/comp.day.pl?var='+variable_entrada+'&level=250mb&iy[1]=&im[1]=&id[1]=&iy[2]=&im[2]=&id[2]=&iy[3]=&im[3]=&id[3]=&iy[4]=&im[4]=&id[4]=&iy[5]=&im[5]=&id[5]=&iy[6]=&im[6]=&id[6]=&iy[7]=&im[7]=&id[7]=&iy[8]=&im[8]=&id[8]=&iy[9]=&im[9]=&id[9]=&iy[10]=&im[10]=&id[10]=&iy[11]=&im[11]=&id[11]=&iy[12]=&im[12]=&id[12]=&iy[13]=&im[13]=&id[13]=&iy[14]=&im[14]=&id[14]=&iy[15]=&im[15]=&id[15]=&iy[16]=&im[16]=&id[16]=&iy[17]=&im[17]=&id[17]=&iy[18]=&im[18]=&id[18]=&iy[19]=&im[19]=&id[19]=&iy[20]=&im[20]=&id[20]=&monr1='+str(mesi)+'&dayr1='+str(diai)+'&monr2='+str(mesf)+'&dayr2='+str(diaf)+'&iyr[1]='+str(aniof)+'&filenamein=&plotlabel=&lag=0&labelc=Color&labels=Shaded&type='+str(tipo)+'&scale=&label=0&cint=&lowr=&highr=&istate=0&proj=ALL&xlat1=&xlat2=&xlon1=&xlon2=&custproj=Cylindrical+Equidistant&level1=1000mb&level2=10mb&Submit=Create+Plot'
        response = urllib.request.urlopen(url)
        data = response.read()      # a `bytes` object

        soup = BeautifulSoup(data,'html.parser') #como es un xml, beautifull tiene un modulo de eso
        link = soup.findAll('img')[-1]['src']
        #Esta es una manera en que podemos obtener el link del .nc que tiene los datos del grafico

        link=list(link)
        link[-3]='n'
        link[-2]='c'
        link[-1]=''
        
        #get nc file save as netcdf
        ruta = "./tmp/"
        urllib.request.urlretrieve('http://www.esrl.noaa.gov'+"".join(link), ruta+variable_salida+'.nc')
    
        

clean()
#initial date

iniy = 2016
inim = 8
inid = 1

#final date
finy = 2016
finm = 8
find = 10
#download: geopotential height, u and v climatology for the selected period
#          geopotential height anomalies

variables = ['Zonal+Wind','Meridional+Wind','Geopotential+Height']

out_var = ['zonalw_climo','meridw_climo','hgt_climo','hgt']

for i in range(0,3):
    if i==2: # download climo and anomaly for geopotential
        descarga_nc(inim,inid,finm,find,finy,variables[i],out_var[i],3)
        descarga_nc(inim,inid,finm,find,finy,variables[i],out_var[i+1],2)
    descarga_nc(inim,inid,finm,find,finy,variables[i],out_var[i],3)


